# 텐서플로 시작하기
- 텐서플포는 수치 계산을 위한 강력한 오픈소스 소프트웨어 라이브러리로 특히 대규모 머신러닝에 맞춰 세밀하게 튜닝되어 있음
- 파이썬으로 수행할 계산 그래프를 정의한 다음, 텐서플로가 최적화된 C++ 코드를 사용해 이 그래프를 효율적으로 실행시킴
- 텐서플로는 CPU, GPU에서 병렬로 실행 가능하며, 분산 컴퓨팅도 지원

In [1]:
import pandas as pd
import numpy as np

In [2]:
# 일관된 출력을 위해 유사난수 초기화
def reset_graph(seed=42):
    tf.reset_default_graph()
    tf.set_random_seed(seed)
    np.random.seed(seed)

## 9.1 설치

## 9.2 첫 번째 계산 그래프를 만들어 세션에서 실행하기

In [3]:
import tensorflow as tf

reset_graph()

x = tf.Variable(3, name="x")
y = tf.Variable(4, name="y")
f = x*x*y + y + 2 # 이 계산 그래프는 뭔가 계산하는 것 같아 보이지만, 실제로 어떤 계산도 수행하고 있지 않음

C:\Users\Yoo\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\Yoo\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\Yoo\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\Yoo\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:529: FutureWarning: Passing (type

Instructions for updating:
Colocations handled automatically by placer.


In [4]:
f

<tf.Tensor 'add_1:0' shape=() dtype=int32>

In [5]:
sess = tf.Session() # 세션 생성
sess.run(x.initializer) # x변수 초기화
sess.run(y.initializer) # y변수 초기화
result = sess.run(f) # f를 평가
print(result) # 출력

sess.close() # 세션 종료

42


In [6]:
with tf.Session() as sess: # with 블록 안에서는 with문에서 선언한 세션이 기본으로 지정됨
    x.initializer.run() # tf.get_default_session().run(x.initializer)를 호출하는 것과 동일
    y.initializer.run() # tf.get_default_session().run(y.initializer)를 호출하는 것과 동일
    result = f.eval() # tf.get_default_session().run(f)를 호출하는 것과 동일
    
result

42

In [7]:
init = tf.global_variables_initializer() # global_variables_initializer ==> 계산 그래프가 실행될 때 모든 변수를 초기화

with tf.Session() as sess:
    init.run() # 실제 모든 변수를 초기화
    result = f.eval()
    
result

42

In [8]:
sess = tf.InteractiveSession() # InteractiveSession ==> 자동으로 자신을 기본 세션으로 지정하는 점만 Session과 다름(with블록 필요 없음)
init.run()
result = f.eval()
print(result)

sess.close() # InteractiveSession의 경우 수동으로 꼭 세션을 종료해줘야 함

42


##### 설명
- 일반적으로 텐서플로 프로그램은 두 부분으로 나뉘는데 첫 부분은 계산 그래프를 만들고(구성 단계), 두 번째 부분은 이 그래프를 실행함(실행 단계)
    - 구성 단계: 훈련에 필요한 계산과 머싱러닝 모델을 표현한 계산 그래프를 생성
    - 실행 단계: 훈련 스텝을 반속해서 평가하고, 모델 파라미터를 점진적으로 개선하기 위해 반복 루프를 수행

## 9.3 계산 그래프 관리

In [9]:
reset_graph()

x1 = tf.Variable(1)
x1.graph is tf.get_default_graph() # 노드를 만들면 자동으로 기본 계산 그래프에 추가됨

True

In [10]:
# 독립적인 계산 그래프를 만들어야 할 경우 
graph = tf.Graph() # 새로운 그래프 객체 생성
with graph.as_default(): # with 블록 안에서 임시로 기본 계산 그래프로 사용
    x2 = tf.Variable(2)

x2.graph is graph # 새로운 그래프 객체에는 포함됨

True

In [11]:
x2.graph is tf.get_default_graph() # 기존 그래프 객체에는 포함 안됨

False

## 9.4 노드 값의 생애주기
- 모든 노드의 값은 계산 그래프 실행 간에 유지되지 않으나 변수값은 예외적으로 그래프 실행 간에도 세션에 의해 유지됨

In [12]:
w = tf.constant(3) # tf.constant로 만들어진 상수는 노드 자체에 값을 지니고 있으며 변경 불가
x = w + 2 # x는 w에 의존
y = x + 5 # y는 x에 의존
z = x * 3

with tf.Session() as sess:
    print(y.eval())  # y가 x에 x가 w에 의존하는 것을 감지하고 w,x,y를 평가해서 10 반환
    print(z.eval())  # y와 같이 w, x를 평가해야 하는 것을 감지하고 재평가를 하여 15 반환(기존 평가 사용 안함)

10
15


In [13]:
with tf.Session() as sess: # 한 번의 그래프 실행에서 y와 z를 모두 평가하여 위의 코드와 같이 w,x를 두 번 평가하지 않게 만들어야 함 
    y_val, z_val = sess.run([y, z]) # y_val, z_val 변수 생성(변수는 초기화될 때 일생이 시작되고 세션이 종료될 때까지 남아 있음)
    print(y_val)  # 10
    print(z_val)  # 15

10
15


## 9.5 텐서플로를 이용한 선형 회귀
- 텐서플로 연산(ops)은 여러 개의 입력을 받아 출력을 만들 수 있음
- 상수와 변수 연산은 입력이 없음(이를 소스 연산이라고도 함)
- 입력과 출력은 텐서라는 다차원 배열(그래서 이름이 텐서플로)이며 넘파이 배열과 비슷하게 데이터 타입과 크기를 가짐(파이썬 API에서는 ndarray 형태)

In [14]:
import numpy as np
from sklearn.datasets import fetch_california_housing # 2장에서 사용한 캘리포니아 주택 가격 데이터

reset_graph()

housing = fetch_california_housing() # 데이터셋 추출
m, n = housing.data.shape # 행, 열의 수 저장
housing_data_plus_bias = np.c_[np.ones((m, 1)), housing.data] # np.c_ ==> 배열들을 2차원으로 변환 후 열로 붙여 배열 만들기(hstack 비슷)

X = tf.constant(housing_data_plus_bias, dtype=tf.float32, name="X") # 텐서플로 상수 노드 X 생성
y = tf.constant(housing.target.reshape(-1, 1), dtype=tf.float32, name="y") # 텐서플로 상수 노드 y 생성
XT = tf.transpose(X) # transpose ==> 텐서를 전치시키는 함수
theta = tf.matmul(tf.matmul(tf.matrix_inverse(tf.matmul(XT, X)), XT), y) # matmul ==> 행렬의 내적 // matrix_inverse ==> 행렬의 역행렬

with tf.Session() as sess:
    theta_value = theta.eval() # transpose, matmul, matrix_inverse 계산 수행

In [15]:
theta_value

array([[-3.6959320e+01],
       [ 4.3698898e-01],
       [ 9.4245886e-03],
       [-1.0791138e-01],
       [ 6.4842808e-01],
       [-3.9986235e-06],
       [-3.7866351e-03],
       [-4.2142656e-01],
       [-4.3467718e-01]], dtype=float32)

In [16]:
# 넘파이와 비교
X = housing_data_plus_bias
y = housing.target.reshape(-1, 1)
theta_numpy = np.linalg.inv(X.T.dot(X)).dot(X.T).dot(y)

print(theta_numpy)

[[-3.69419202e+01]
 [ 4.36693293e-01]
 [ 9.43577803e-03]
 [-1.07322041e-01]
 [ 6.45065694e-01]
 [-3.97638942e-06]
 [-3.78654265e-03]
 [-4.21314378e-01]
 [-4.34513755e-01]]


In [17]:
# 사이킷런과 비교
from sklearn.linear_model import LinearRegression
lin_reg = LinearRegression()
lin_reg.fit(housing.data, housing.target.reshape(-1, 1))

print(np.r_[lin_reg.intercept_.reshape(-1, 1), lin_reg.coef_.T])

[[-3.69419202e+01]
 [ 4.36693293e-01]
 [ 9.43577803e-03]
 [-1.07322041e-01]
 [ 6.45065694e-01]
 [-3.97638942e-06]
 [-3.78654265e-03]
 [-4.21314378e-01]
 [-4.34513755e-01]]


## 9.6 경사 하강법 구현
- 1. 그래디언트를 수동으로 계산
- 2. 텐서플로의 자동 미분 기능을 이용해 그래디언트를 자동으로 계산
- 3. 내장된 옵티마이저를 사용하여 계산
- 경사 하강법은 먼저 특성 벡터의 스케일을 조정해야 하는데, 텐서플로를 사용해 할 수 있지만 그냥 여기서는 사이킷런을 사용

In [18]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaled_housing_data = scaler.fit_transform(housing.data)
scaled_housing_data_plus_bias = np.c_[np.ones((m, 1)), scaled_housing_data] # 벡터의 스케일 조정

In [19]:
print(scaled_housing_data_plus_bias.mean(axis=0))
print(scaled_housing_data_plus_bias.mean(axis=1))
print(scaled_housing_data_plus_bias.mean())
print(scaled_housing_data_plus_bias.shape)

[ 1.00000000e+00  6.60969987e-17  5.50808322e-18  6.60969987e-17
 -1.06030602e-16 -1.10161664e-17  3.44255201e-18 -1.07958431e-15
 -8.52651283e-15]
[ 0.38915536  0.36424355  0.5116157  ... -0.06612179 -0.06360587
  0.01359031]
0.11111111111111005
(20640, 9)


#### 9.6.1 직접 그래디언트 계산

In [20]:
reset_graph()

n_epochs = 1000
learning_rate = 0.01

X = tf.constant(scaled_housing_data_plus_bias, dtype=tf.float32, name="X") # 텐서플로 상수 노드 X 생성
y = tf.constant(housing.target.reshape(-1, 1), dtype=tf.float32, name="y") # 텐서플로 상수 노드 y 생성
theta = tf.Variable(tf.random_uniform([n + 1, 1], -1.0, 1.0, seed=42), name="theta") # 텐서플로 변수 노드 theta 생성
# random_uniform ==> 난수를 담은 텐서를 생성하는 노드를 그래프에 생성(넘파이 rand와 같이 크기와 범위 설정 받음)
y_pred = tf.matmul(X, theta, name="predictions") # 예측값
error = y_pred - y # 오차값
mse = tf.reduce_mean(tf.square(error), name="mse") # reduce_mean ==> 특정 차원을 제거 후 평균을 계산
gradients = 2/m * tf.matmul(tf.transpose(X), error) # 그래디언트 계산
training_op = tf.assign(theta, theta - learning_rate * gradients) 
# assign ==> 변수에 새로운 값을 할당하는 노드를 생성(theta = ~~라고 쓰면 theta는 더이상 변수가 아니라 연산의 출력을 가르키는 텐서가 됨)

init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init) # 실제 모든 변수를 초기화

    for epoch in range(n_epochs):
        if epoch % 100 == 0: # 100번째 마다 MSE 출력
            print("에포크", epoch, "MSE =", mse.eval())
        sess.run(training_op) # training_op를 평가
    
    best_theta = theta.eval()
    
print("best_theta:")
print(best_theta)

에포크 0 MSE = 9.161542
에포크 100 MSE = 0.71450055
에포크 200 MSE = 0.56670487
에포크 300 MSE = 0.55557173
에포크 400 MSE = 0.5488112
에포크 500 MSE = 0.5436363
에포크 600 MSE = 0.53962904
에포크 700 MSE = 0.5365092
에포크 800 MSE = 0.53406775
에포크 900 MSE = 0.5321473
best_theta:
[[ 2.0685523 ]
 [ 0.8874027 ]
 [ 0.14401656]
 [-0.34770885]
 [ 0.36178368]
 [ 0.00393811]
 [-0.04269556]
 [-0.66145283]
 [-0.6375278 ]]


#### 9.6.2 자동 미분 사용

In [21]:
# 이전의 코드와 gradients 부분만 다름
reset_graph()

n_epochs = 1000
learning_rate = 0.01

X = tf.constant(scaled_housing_data_plus_bias, dtype=tf.float32, name="X")
y = tf.constant(housing.target.reshape(-1, 1), dtype=tf.float32, name="y")
theta = tf.Variable(tf.random_uniform([n + 1, 1], -1.0, 1.0, seed=42), name="theta")
y_pred = tf.matmul(X, theta, name="predictions")
error = y_pred - y
mse = tf.reduce_mean(tf.square(error), name="mse")
gradients = tf.gradients(mse, [theta])[0]
# gradients ==> 하나의 연산 mse와 변수 리스트(여기서는 theta 하나)를 받아 각 변수에 대한 연산의 그래디언트를 계산하는 연산을 만듬
training_op = tf.assign(theta, theta - learning_rate * gradients)

init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)

    for epoch in range(n_epochs):
        if epoch % 100 == 0:
            print("에포크", epoch, "MSE =", mse.eval())
        sess.run(training_op)
    
    best_theta = theta.eval()

print("best_theta:")
print(best_theta)

에포크 0 MSE = 9.161542
에포크 100 MSE = 0.7145004
에포크 200 MSE = 0.56670487
에포크 300 MSE = 0.55557173
에포크 400 MSE = 0.5488112
에포크 500 MSE = 0.5436363
에포크 600 MSE = 0.53962904
에포크 700 MSE = 0.5365092
에포크 800 MSE = 0.53406775
에포크 900 MSE = 0.5321473
best_theta:
[[ 2.0685525 ]
 [ 0.8874027 ]
 [ 0.14401658]
 [-0.34770882]
 [ 0.36178368]
 [ 0.00393811]
 [-0.04269556]
 [-0.6614528 ]
 [-0.6375277 ]]


In [22]:
def my_func(a, b): # a와 b에 대한 다음 함수의 편도함수 구하는 함수
    z = 0
    for i in range(100):
        z = a * np.cos(z + i) + z * np.sin(b - i)
    return z

my_func(0.2, 0.3)

-0.21253923284754914

In [23]:
# a(0.2), b(0.3)일 때의 함수 값을 계산하고 그 다음 a와 b에 대한 편미분을 구하기
reset_graph()

a = tf.Variable(0.2, name="a") # 변수 노드
b = tf.Variable(0.3, name="b") # 변수 노드
z = tf.constant(0.0, name="z0") # 상수 노드
for i in range(100):
    z = a * tf.cos(z + i) + z * tf.sin(b - i)

grads = tf.gradients(z, [a, b]) # 텐서플로에서는 자동 미분 기능이 있어 편도함수 구하는 함수를 별도로 안만들어도 됨
init = tf.global_variables_initializer()

with tf.Session() as sess:
    init.run() # 실제 모든 변수를 초기화
    print(z.eval()) # 함수 값
    print(sess.run(grads)) # 편미분 값

-0.21253741
[-1.1388494, 0.19671395]


##### 대표적인 그래디언트 자동 계산 방식
- 1. 수치미분
- 2. 기호 미분
- 3. 전진 모드 자동 미분
- 4. 후진 모드 자동 미분(텐서플로에서 사용하는 방식)

#### 9.6.3 옵티마이저 사용

In [24]:
# 이전의 코드와 gradients => optimizer, training_op 부분만 다름
reset_graph()

n_epochs = 1000
learning_rate = 0.01

X = tf.constant(scaled_housing_data_plus_bias, dtype=tf.float32, name="X")
y = tf.constant(housing.target.reshape(-1, 1), dtype=tf.float32, name="y")
theta = tf.Variable(tf.random_uniform([n + 1, 1], -1.0, 1.0, seed=42), name="theta")
y_pred = tf.matmul(X, theta, name="predictions")
error = y_pred - y
mse = tf.reduce_mean(tf.square(error), name="mse")
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate) # 경사 하강법 옵티마이저
training_op = optimizer.minimize(mse)

init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)

    for epoch in range(n_epochs):
        if epoch % 100 == 0:
            print("에포크", epoch, "MSE =", mse.eval())
        sess.run(training_op)
    
    best_theta = theta.eval()

print("best_theta:")
print(best_theta)

에포크 0 MSE = 9.161542
에포크 100 MSE = 0.7145004
에포크 200 MSE = 0.56670487
에포크 300 MSE = 0.55557173
에포크 400 MSE = 0.5488112
에포크 500 MSE = 0.5436363
에포크 600 MSE = 0.53962904
에포크 700 MSE = 0.5365092
에포크 800 MSE = 0.53406775
에포크 900 MSE = 0.5321473
best_theta:
[[ 2.0685525 ]
 [ 0.8874027 ]
 [ 0.14401658]
 [-0.34770882]
 [ 0.36178368]
 [ 0.00393811]
 [-0.04269556]
 [-0.6614528 ]
 [-0.6375277 ]]


In [25]:
# 모멘텀 옵티마이저 사용(11장 참고)
reset_graph()

n_epochs = 1000
learning_rate = 0.01

X = tf.constant(scaled_housing_data_plus_bias, dtype=tf.float32, name="X")
y = tf.constant(housing.target.reshape(-1, 1), dtype=tf.float32, name="y")
theta = tf.Variable(tf.random_uniform([n + 1, 1], -1.0, 1.0, seed=42), name="theta")
y_pred = tf.matmul(X, theta, name="predictions")
error = y_pred - y
mse = tf.reduce_mean(tf.square(error), name="mse")
optimizer = tf.train.MomentumOptimizer(learning_rate=learning_rate, momentum=0.9) # 옵티마이저만 바꾸면 됨
training_op = optimizer.minimize(mse)

init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)

    for epoch in range(n_epochs):
        sess.run(training_op)
    
    best_theta = theta.eval()

print("best_theta:")
print(best_theta)

best_theta:
[[ 2.068558  ]
 [ 0.8296286 ]
 [ 0.11875337]
 [-0.26554456]
 [ 0.3057109 ]
 [-0.00450251]
 [-0.03932662]
 [-0.89986444]
 [-0.87052065]]


## 9.7 훈련 알고리즘에 데이터 주입
- 미니배치 경사 하강법을 구현하려 할 때는 X, y를 매 반복마다 변경해야 하지만, 이는 플레이스 홀더 노드를 통해 간단히 해결 가능

In [26]:
# 플레이스 홀더 예시
reset_graph()

A = tf.placeholder(tf.float32, shape=(None, 3)) 
# placeholder ==> 실제로 아무 계산도 하지 않는 특수한 노드로 주입한 데이터만을 출력함(데이터 타입과 크기를 지정해야 함(None ==> 무한대))
B = A + 5
with tf.Session() as sess:
    B_val_1 = B.eval(feed_dict={A: [[1, 2, 3]]}) # feed_dict ==> placeholder에 값 전달
    B_val_2 = B.eval(feed_dict={A: [[4, 5, 6], [7, 8, 9]]})

print(B_val_1)
print('')
print(B_val_2)

[[6. 7. 8.]]

[[ 9. 10. 11.]
 [12. 13. 14.]]


In [27]:
# 미니배치 경사 하강법 구현
reset_graph()

n_epochs = 1000
learning_rate = 0.01

X = tf.placeholder(tf.float32, shape=(None, n + 1), name="X") # 플레이스 홀더 노드 X 생성
y = tf.placeholder(tf.float32, shape=(None, 1), name="y") # 플레이스 홀더 노드 y 생성
theta = tf.Variable(tf.random_uniform([n + 1, 1], -1.0, 1.0, seed=42), name="theta") # 변수 노드 theta 생성
y_pred = tf.matmul(X, theta, name="predictions") # 예측값
error = y_pred - y # 오차값
mse = tf.reduce_mean(tf.square(error), name="mse") # mse 계산
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate) # 경사 하강법 옵티마이저
training_op = optimizer.minimize(mse) # mse 최소값 저장

init = tf.global_variables_initializer()

n_epochs = 10
batch_size = 100
n_batches = int(np.ceil(m / batch_size)) # ceil ==> 값 올림 처리 // 행의 수 % batch size

def fetch_batch(epoch, batch_index, batch_size):
    np.random.seed(epoch * n_batches + batch_index)  # 지정 값마다 난수 고정
    indices = np.random.randint(m, size=batch_size)  # 0~행의 수까지 랜덤 정수를 batch size만큼 생성
    X_batch = scaled_housing_data_plus_bias[indices] # 사이킷런의 표준화 후 값 X_batch 지정
    y_batch = housing.target.reshape(-1, 1)[indices] # 타겟 값 y_batch 지정
    return X_batch, y_batch

with tf.Session() as sess:
    sess.run(init)

    for epoch in range(n_epochs): # 10회 반복
        for batch_index in range(n_batches): # (행의 수 % 100)의 올림 값만큼 반복
            X_batch, y_batch = fetch_batch(epoch, batch_index, batch_size)
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch}) # fetch_batch 반환 값 플레이스 홀더에 주입 후 training_op 계산

    best_theta = theta.eval()
    
print("best_theta:")
print(best_theta)

best_theta:
[[ 2.0703337 ]
 [ 0.8637145 ]
 [ 0.12255151]
 [-0.31211874]
 [ 0.38510373]
 [ 0.00434168]
 [-0.01232954]
 [-0.83376896]
 [-0.8030471 ]]


## 9.8 모델 저장과 복원
- 모델을 훈련시키고 나면 필요할 때 다시 쓸 수 있도록 모델 파라미터를 저장이 필요하며, 체크포인트를 저장해두면 중간에 문제가 생겨도 다시 이어나갈 수 있음

In [28]:
reset_graph()

n_epochs = 1000
learning_rate = 0.01

X = tf.constant(scaled_housing_data_plus_bias, dtype=tf.float32, name="X")
y = tf.constant(housing.target.reshape(-1, 1), dtype=tf.float32, name="y")
theta = tf.Variable(tf.random_uniform([n + 1, 1], -1.0, 1.0, seed=42), name="theta")
y_pred = tf.matmul(X, theta, name="predictions")
error = y_pred - y
mse = tf.reduce_mean(tf.square(error), name="mse")
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
training_op = optimizer.minimize(mse)

init = tf.global_variables_initializer()
saver = tf.train.Saver() # Saver ==> 저장 노드

with tf.Session() as sess:
    sess.run(init)

    for epoch in range(n_epochs):
        if epoch % 100 == 0:
            print("에포크", epoch, "MSE =", mse.eval())
            save_path = saver.save(sess, "/tmp/my_model.ckpt")
        sess.run(training_op)
    
    best_theta = theta.eval()
    save_path = saver.save(sess, "/tmp/my_model_final.ckpt") # saver 노드를 추가 후 save 메서드(체크포인트, 경로) 지정 후 호출하면 됨
    
print("best_theta:")
print(best_theta)

에포크 0 MSE = 9.161542
에포크 100 MSE = 0.7145004
에포크 200 MSE = 0.56670487
에포크 300 MSE = 0.55557173
에포크 400 MSE = 0.5488112
에포크 500 MSE = 0.5436363
에포크 600 MSE = 0.53962904
에포크 700 MSE = 0.5365092
에포크 800 MSE = 0.53406775
에포크 900 MSE = 0.5321473
best_theta:
[[ 2.0685525 ]
 [ 0.8874027 ]
 [ 0.14401658]
 [-0.34770882]
 [ 0.36178368]
 [ 0.00393811]
 [-0.04269556]
 [-0.6614528 ]
 [-0.6375277 ]]


In [29]:
with tf.Session() as sess:
    saver.restore(sess, "/tmp/my_model_final.ckpt") # saver 노드 생성 후 변수 초기화(init)대신 restore메서드(체크포인트, 경로)로 호출
    best_theta_restored = theta.eval()

Instructions for updating:
Use standard file APIs to check for files with this prefix.
INFO:tensorflow:Restoring parameters from /tmp/my_model_final.ckpt


In [30]:
np.allclose(best_theta, best_theta_restored) # 동일 여부 확인

True

In [31]:
# saver은 기본적으로 모든 변수를 각자의 이름으로 저장하고 복원
saver = tf.train.Saver({"weights": theta}) # 만약 변수 이름을 바꿔서 저장하고 복원하고 싶은 경우

In [32]:
# save 메서드는 기본적으로 .meta 확장자를 가진 동일 이름의 두 번째 파일에 그래프를 저장하므로 그래프도 복원 가능
reset_graph()

saver = tf.train.import_meta_graph("/tmp/my_model_final.ckpt.meta")  # 그래프 구조를 로드
theta = tf.get_default_graph().get_tensor_by_name("theta:0")

with tf.Session() as sess:
    saver.restore(sess, "/tmp/my_model_final.ckpt")  # 그래프 상태를 로드합니다.
    best_theta_restored = theta.eval()

INFO:tensorflow:Restoring parameters from /tmp/my_model_final.ckpt


In [33]:
np.allclose(best_theta, best_theta_restored) # 동일 여부 확인

True

## 9.9 텐서보드로 그래프와 학습 곡선 시각화하기
- 주피터 노트북에서 그래프를 나타내기 위해 https://tensorboard.appspot.com/ 에 서비스 중인 텐서보드 서버를 사용(인터넷 연결 필수)

In [34]:
# 로드 디렉터리 설정(프로그램 실행할 때마다 다른 로그 디렉터리 사용하지 않으면 만들어진 통계가 계속 합쳐쳐서 엉망이 됨)
reset_graph()

from datetime import datetime

now = datetime.utcnow().strftime("%Y%m%d%H%M%S") # 현재 시간
root_logdir = "tf_logs"
logdir = "{}/run-{}/".format(root_logdir, now) # 현재 시간으로 로그 디렉터리 설정

In [35]:
n_epochs = 1000
learning_rate = 0.01

X = tf.placeholder(tf.float32, shape=(None, n + 1), name="X")
y = tf.placeholder(tf.float32, shape=(None, 1), name="y")
theta = tf.Variable(tf.random_uniform([n + 1, 1], -1.0, 1.0, seed=42), name="theta")
y_pred = tf.matmul(X, theta, name="predictions")
error = y_pred - y
mse = tf.reduce_mean(tf.square(error), name="mse")
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
training_op = optimizer.minimize(mse)

init = tf.global_variables_initializer()

mse_summary = tf.summary.scalar('MSE', mse) 
# mse 값을 평가하고 그것을 서머리라고 부르는 텐서보드가 인식하는 이진 로그 문자열에 쓰기 위한 노드를 그래프에 추가
file_writer = tf.summary.FileWriter(logdir, tf.get_default_graph())
# FileWriter 객체를 만들어 로그 디렉터리에 있는 로그 파일에 서머리를 기록(로그 디렉터리 경로, 시각화하고자 하는 계산 그래프)

n_epochs = 10
batch_size = 100
n_batches = int(np.ceil(m / batch_size))

with tf.Session() as sess:
    sess.run(init)

    for epoch in range(n_epochs):
        for batch_index in range(n_batches):
            X_batch, y_batch = fetch_batch(epoch, batch_index, batch_size)
            if batch_index % 10 == 0:
                summary_str = mse_summary.eval(feed_dict={X: X_batch, y: y_batch}) # mse_summary를 정기적으로 평가
                step = epoch * n_batches + batch_index
                file_writer.add_summary(summary_str, step) # 만들어진 서머리는 file_writer를 사용하여 이벤트 파일에 기록
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})

    best_theta = theta.eval()  

In [36]:
file_writer.close() # FileWriter 객체 닫기

In [37]:
best_theta

array([[ 2.0703337 ],
       [ 0.8637145 ],
       [ 0.12255151],
       [-0.31211874],
       [ 0.38510373],
       [ 0.00434168],
       [-0.01232954],
       [-0.83376896],
       [-0.8030471 ]], dtype=float32)

#### 시각화 부분 페이지 에러로 생략(추후 시도 필요...)

## 9.10 이름 범위
- 신경망처럼 매우 복잡한 모델을 다룰 경우 계산 그래프가 수천 개의 노드로 인해 어질러지기 쉬우므로 이름 범위를 만들어 관련 있는 노드들을 그룹으로 묶어야 함

In [38]:
reset_graph()

now = datetime.utcnow().strftime("%Y%m%d%H%M%S")
root_logdir = "tf_logs"
logdir = "{}/run-{}/".format(root_logdir, now)

n_epochs = 1000
learning_rate = 0.01

X = tf.placeholder(tf.float32, shape=(None, n + 1), name="X")
y = tf.placeholder(tf.float32, shape=(None, 1), name="y")
theta = tf.Variable(tf.random_uniform([n + 1, 1], -1.0, 1.0, seed=42), name="theta")
y_pred = tf.matmul(X, theta, name="predictions")

with tf.name_scope("loss") as scope: # loss 범위 안에 mse, error를 정의
    error = y_pred - y
    mse = tf.reduce_mean(tf.square(error), name="mse")
    
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
training_op = optimizer.minimize(mse)

init = tf.global_variables_initializer()

mse_summary = tf.summary.scalar('MSE', mse)
file_writer = tf.summary.FileWriter(logdir, tf.get_default_graph())

n_epochs = 10
batch_size = 100
n_batches = int(np.ceil(m / batch_size))

with tf.Session() as sess:
    sess.run(init)

    for epoch in range(n_epochs):
        for batch_index in range(n_batches):
            X_batch, y_batch = fetch_batch(epoch, batch_index, batch_size)
            if batch_index % 10 == 0:
                summary_str = mse_summary.eval(feed_dict={X: X_batch, y: y_batch})
                step = epoch * n_batches + batch_index
                file_writer.add_summary(summary_str, step)
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})

    best_theta = theta.eval()

file_writer.flush()
file_writer.close()
print("best_theta:")
print(best_theta)

best_theta:
[[ 2.0703337 ]
 [ 0.8637145 ]
 [ 0.12255151]
 [-0.31211874]
 [ 0.38510373]
 [ 0.00434168]
 [-0.01232954]
 [-0.83376896]
 [-0.8030471 ]]


In [39]:
print(error.op.name, mse.op.name) # loss 범위 안에 있는 모든 연산의 이름은 'loss/' 접두사가 붙음

loss/sub loss/mse


In [40]:
# 변수, 범위 이름 예시
reset_graph()

a1 = tf.Variable(0, name="a")      # name == "a"
a2 = tf.Variable(0, name="a")      # name == "a_1"

with tf.name_scope("param"):       # name == "param"
    a3 = tf.Variable(0, name="a")  # name == "param/a"

with tf.name_scope("param"):       # name == "param_1"
    a4 = tf.Variable(0, name="a")  # name == "param_1/a"

for node in (a1, a2, a3, a4):
    print(node.op.name) # 범위로 묶은 연산들은 접두사가 붙음(같은 이름의 변수, 범위는 뒤에 고유 숫자가 붙음)

a
a_1
param/a
param_1/a


## 9.11 모듈화
- 반복된 작업을 수행해야 하는 경우

In [41]:
# 두 개의 ReLU(h(X) = max(X*w+b, 0)) 출력을 더하는 그래프 만들기 => 반복이 많음(즉, 유지 보수하기 어렵고 에러가 발생하기 쉬움)
reset_graph()

n_features = 3
X = tf.placeholder(tf.float32, shape=(None, n_features), name="X") # 플레이스 홀더 노드 X 생성

w1 = tf.Variable(tf.random_normal((n_features, 1)), name="weights1") # 변수 노드 w1 생성
w2 = tf.Variable(tf.random_normal((n_features, 1)), name="weights2") # 변수 노드 w2 생성
b1 = tf.Variable(0.0, name="bias1") # 변수 노드 b1 생성
b2 = tf.Variable(0.0, name="bias2") # 변수 노드 b2 생성

z1 = tf.add(tf.matmul(X, w1), b1, name="z1")
z2 = tf.add(tf.matmul(X, w2), b2, name="z2")

relu1 = tf.maximum(z1, 0., name="relu1")
relu2 = tf.maximum(z1, 0., name="relu2")  # z1 -> z2로 고쳐져야함(이런 방식은 실수가 많고 에러가 발생하기 쉽다는 것을 표현하기 위한 것)

output = tf.add(relu1, relu2, name="output")

In [42]:
# DRY(Don't Repeat Yourself) 원칙을 이용한 두 개의 ReLU(h(X) = max(X*w+b, 0)) 출력을 더하는 그래프 만들기
reset_graph()

def relu(X): # ReLU함수
    w_shape = (int(X.get_shape()[1]), 1) # (X의 열 수, 1)의 형태
    w = tf.Variable(tf.random_normal(w_shape), name="weights") # 변수 노드 w 생성
    b = tf.Variable(0.0, name="bias") # 변수 노드 b 생성
    z = tf.add(tf.matmul(X, w), b, name="z")
    return tf.maximum(z, 0., name="relu")

n_features = 3
X = tf.placeholder(tf.float32, shape=(None, n_features), name="X") # 플레이스 홀더 노드 X 생성
relus = [relu(X) for i in range(5)] # 다섯 번 반복
output = tf.add_n(relus, name="output") # 중복된 이름의 노드들은 뒤의 고유 숫자가 붙음

file_writer = tf.summary.FileWriter("logs/relu1", tf.get_default_graph())

In [43]:
# DRY원칙과 이름 범위를 이용한 두 개의 ReLU(h(X) = max(X*w+b, 0)) 출력을 더하는 그래프 만들기
reset_graph()

def relu(X):
    with tf.name_scope("relu"): # relu 라는 이름 범위로 묶기
        w_shape = (int(X.get_shape()[1]), 1)
        w = tf.Variable(tf.random_normal(w_shape), name="weights")
        b = tf.Variable(0.0, name="bias")
        z = tf.add(tf.matmul(X, w), b, name="z")
        return tf.maximum(z, 0., name="max")
    
n_features = 3
X = tf.placeholder(tf.float32, shape=(None, n_features), name="X")
relus = [relu(X) for i in range(5)]
output = tf.add_n(relus, name="output") # 중복된 이름 범위들도 뒤의 고유 숫자가 붙음

file_writer = tf.summary.FileWriter("logs/relu2", tf.get_default_graph())
file_writer.close()

## 9.12 변수 공유
- 그래프의 여러 구성 요소 간에 변수를 공유하고 싶은 경우

In [44]:
# 변수를 함수에 전달
reset_graph()

def relu(X, threshold): # 기존엔 X만 받았으나 여기선 threshold도 받음
    with tf.name_scope("relu"):
        w_shape = (int(X.get_shape()[1]), 1)
        w = tf.Variable(tf.random_normal(w_shape), name="weights")
        b = tf.Variable(0.0, name="bias")
        z = tf.add(tf.matmul(X, w), b, name="z")
        return tf.maximum(z, threshold, name="max")

threshold = tf.Variable(0.0, name="threshold") # 변수 노드 threshold 생성
X = tf.placeholder(tf.float32, shape=(None, n_features), name="X")
relus = [relu(X, threshold) for i in range(5)]
output = tf.add_n(relus, name="output")

In [45]:
# 함수 내에서 공유 변수 만들기
reset_graph()

def relu(X):
    with tf.name_scope("relu"):
        if not hasattr(relu, "threshold"):
            relu.threshold = tf.Variable(0.0, name="threshold") # threshold 공유 변수가 없는 경우 변수 노드 threshold 생성
        w_shape = int(X.get_shape()[1]), 1
        w = tf.Variable(tf.random_normal(w_shape), name="weights")
        b = tf.Variable(0.0, name="bias")
        z = tf.add(tf.matmul(X, w), b, name="z")
        return tf.maximum(z, relu.threshold, name="max")
    
X = tf.placeholder(tf.float32, shape=(None, n_features), name="X")
relus = [relu(X) for i in range(5)]
output = tf.add_n(relus, name="output")

In [46]:
# get_variable 함수 사용 예시들
reset_graph()

with tf.variable_scope("relu"): # relu/threshold 변수 생성
    threshold = tf.get_variable("threshold", shape=(), initializer=tf.constant_initializer(0.0))
    

with tf.variable_scope("relu", reuse=True): # reuse ==> 이미 변수가 생성됐다면 재사용(한 번 True가 되면 블록 안에서 False 못 바꿈)
    threshold = tf.get_variable("threshold") # 재사용이므로 초기값, 크기 필요 없음
    

with tf.variable_scope("relu") as scope: # relu/threshold 변수 가져오며, 존재하지 않거나 get_variable로 생성되지 않은 경우 예외 발생
    scope.reuse_variables()
    threshold = tf.get_variable("threshold")

In [47]:
# 함수 밖에서 공유 변수 생성 후 재사용 설정
reset_graph()

def relu(X):
    with tf.variable_scope("relu", reuse=True): # reuse=True로 재사용으로 설정한 것
        threshold = tf.get_variable("threshold")
        w_shape = int(X.get_shape()[1]), 1
        w = tf.Variable(tf.random_normal(w_shape), name="weights")
        b = tf.Variable(0.0, name="bias")
        z = tf.add(tf.matmul(X, w), b, name="z")
        return tf.maximum(z, threshold, name="max")

X = tf.placeholder(tf.float32, shape=(None, n_features), name="X")
with tf.variable_scope("relu"): # 함수 정의 후 get_variable로 relu/threshold 변수 생성
    threshold = tf.get_variable("threshold", shape=(), initializer=tf.constant_initializer(0.0))
relus = [relu(X) for relu_index in range(5)] # 5회 반복
output = tf.add_n(relus, name="output")

In [48]:
file_writer = tf.summary.FileWriter("logs/relu6", tf.get_default_graph())
file_writer.close()

In [49]:
# 함수 내에서 공유 변수 생성 후 재사용 설정
reset_graph()

def relu(X):
    with tf.variable_scope("relu"):
        threshold = tf.get_variable("threshold", shape=(), initializer=tf.constant_initializer(0.0))
        w_shape = (int(X.get_shape()[1]), 1)
        w = tf.Variable(tf.random_normal(w_shape), name="weights")
        b = tf.Variable(0.0, name="bias")
        z = tf.add(tf.matmul(X, w), b, name="z")
        return tf.maximum(z, threshold, name="max")

X = tf.placeholder(tf.float32, shape=(None, n_features), name="X")

with tf.variable_scope("", default_name="") as scope:
    first_relu = relu(X)     # 공유 변수를 만든 후
    scope.reuse_variables()  # 재사용합니다.
    relus = [first_relu] + [relu(X) for i in range(4)]

output = tf.add_n(relus, name="output")

file_writer = tf.summary.FileWriter("logs/relu8", tf.get_default_graph())
file_writer.close()

In [50]:
# 함수 내에서 공유 변수 생성 후 재사용 설정2
reset_graph()

def relu(X):
    threshold = tf.get_variable("threshold", shape=(), initializer=tf.constant_initializer(0.0))
    w_shape = (int(X.get_shape()[1]), 1)
    w = tf.Variable(tf.random_normal(w_shape), name="weights")
    b = tf.Variable(0.0, name="bias")
    z = tf.add(tf.matmul(X, w), b, name="z")
    return tf.maximum(z, threshold, name="max")

X = tf.placeholder(tf.float32, shape=(None, n_features), name="X")
relus = []
for relu_index in range(5):
    with tf.variable_scope("relu", reuse=(relu_index >= 1)) as scope: # 처음만 변수 생성 후 이후 재사용
        relus.append(relu(X))
output = tf.add_n(relus, name="output")

In [51]:
file_writer = tf.summary.FileWriter("logs/relu9", tf.get_default_graph())
file_writer.close()

## 추가. 

In [52]:
reset_graph()

with tf.variable_scope("my_scope"):
    x0 = tf.get_variable("x", shape=(), initializer=tf.constant_initializer(0.))
    x1 = tf.Variable(0., name="x")
    x2 = tf.Variable(0., name="x")

with tf.variable_scope("my_scope", reuse=True):
    x3 = tf.get_variable("x")
    x4 = tf.Variable(0., name="x")

with tf.variable_scope("", default_name="", reuse=True):
    x5 = tf.get_variable("my_scope/x")

print("x0:", x0.op.name)
print("x1:", x1.op.name) # 변수애 고유번호 1 붙음
print("x2:", x2.op.name) # 변수에 고유번호 2 붙음
print("x3:", x3.op.name) # x0 공유 변수 재사용
print("x4:", x4.op.name) # 이름 범위에 고유번호 1 붙음
print("x5:", x5.op.name) # x3 공유 변수 재사용
print(x0 is x3 and x3 is x5)

x0: my_scope/x
x1: my_scope/x_1
x2: my_scope/x_2
x3: my_scope/x
x4: my_scope_1/x
x5: my_scope/x
True


##### 설명
- 1. 첫 번째 variable_scope() 블럭은 이름이 my_scope/x인 공유 변수 x0를 만듭니다. 공유 변수 이외의 모든 연산에 대해서는 (공유되지 않는 변수를 포함하여) 변수 범위가 일반적인 이름 범위처럼 작동합니다. 그래서 두 변수 x1과 x2에 접두사 my_scope/가 붙습니다. 하지만 텐서플로는 이름을 고유하게 만들기 위해 my_scope/x_1, my_scope/x_2처럼 인덱스를 추가시킵니다.

- 2. 두 번째 variable_scope() 블럭은 my_scope 범위에 있는 공유 변수를 재사용합니다. 그래서 x0 is x3가 참입니다. 여기에서도 공유 변수를 제외한 모든 연산은 이름 범주와 같이 작동합니다. 첫 번째 블럭과 다르기 때문에 텐서플로가 고유한 범주 이름을 만듭니다(my_scope_1). 변수 x4의 이름은 my_scope_1/x가 됩니다.

- 3. 세 번째 블럭은 공유 변수 my_scope/x를 다루는 다른 방식을 보여 줍니다. 루트 범위(이름이 빈 문자열입니다)에서 variable_scope()를 만들고 공유 변수의 전체 이름(즉, "my_scope/x")으로 get_variable()을 호출합니다.

In [53]:
reset_graph()

text = np.array("Do you want some café?".split()) # 띄어쓰기로 문자열 분리
text_tensor = tf.constant(text) # 상수 노드

with tf.Session() as sess:
    print(text_tensor.eval()) # 문자열 분리 시행

[b'Do' b'you' b'want' b'some' b'caf\xc3\xa9?']


## 9.13 연습문제

#### 1. 계산을 직접 실행하지 않고 계산 그래프를 만드는 주요 장점과 단점은 무엇인가요?
##### 장점
- 텐서플로가 자동으로 그래디언트를 계산할 수 있음(후진 모드 자동 미분을 사용하여)
- 텐서플로가 여러 스레드에서 연산을 병렬로 실행할 수 있음
- 동일한 모델을 여러 장치에 걸쳐 실행시키기 편리함
- 내부 구조를 살피기 쉬움(예를 들어 텐서보드에서 모델을 시각화할 수 있음)

##### 단점
- 익숙하게 다룰려면 시간이 필요
- 단계별 디버깅 수행이 어려움

#### 2. a_val = a.eval(session=sess)와 a_val = sess.run(a)는 동일한 문장인가요?
- 완전히 동일

#### 3. a_val, b_val = a.eval(session=sess), b.eval(session=sess)와 a_val, b_val = sess.run([a,b])는 동일한 문장인가요?
- 두 문장은 동일하지 않는다. 첫 번째 문장은 그래프를 두 번 실행하지만, 두 번째 문장은 그래프를 한 번만 실행한다. 이 연산(또는 의존하는 다른 연산)이 부수효과를 일으키면(예를 들어 변수가 수정되거나, 큐에 아이템이 추가되거나, 리더가 파일을 읽으면) 결과가 달라질 것입니다. 만약 부수효과가 없다면 두 문장은 동일한 결과를 반환하지만 두 번째 문장이 첫 번째 문장보다 속도가 더 빠를 것입니다.

#### 4. 같은 세션에서 두 개의 그래프를 실행할 수 있나요?
- 아니오. 같은 세션에서 두 개의 그래프를 실행할 수 없습니다. 먼저 두 개의 그래프를 하나의 그래프로 합쳐야 합니다.

#### 5. 만약 변수 w를 가진 그래프 g를 만들고 스레드 두 개를 시작해 각 스레드에서 동일한 그래프 g를 사용하는 세션을 열면, 각 세션은 변수 w를 따로 가지게 될까요? 아니면 공유할까요?
- 로컬 텐서플로에서는 세션이 변숫값을 관리하므로 만약 변수 w를 가진 그래프 g를 만들고 동일한 그래프 g를 사용하는 두 개의 스레드를 시작해 각 스레드에서 로컬 세션을 열면 각 세션은 변수 w의 복사본을 각자 가지게 될 것입니다. 그러나 분산 텐서플로에서는 변수값이 클러스터에 의해 관리되는 컨테이너에 저장됩니다. 그러므로 두 개의 텐서플로에서는 변수값이 클러스터에 의해 관리되는 컨테이너에 저장됩니다. 그러므로 두 개의 세션이 같은 클러스터에 접속하여 같은 컨테이너를 사용하면 동일한 변수 w의 값을 공유할 것입니다.

#### 6. 변수는 언제 초기화되고 언제 소멸되나요?
- 변수는 초기화 함수가 호출될 때 초기화되고, 세션이 종료될 때 소멸됩니다. 분산 텐서플로에서는 변수가 클러스터에 있는 컨테이너에 존재하기 때문에 세션을 종료해도 변수가 소멸되지 않으며 변수를 삭제하려면 컨터이너를 리셋해야 합니다.

#### 7. 플레이스 홀더와 변수의 차이점은 무엇인가요?
- 변수는 값을 가진 연산입니다. 변수를 실행하면 값이 반환됩니다. 변수는 실행하기 전에 초기화해야 합니다. 또한 변수의 값을 바꿀 수 있습니다(예를 들면 할당 연산을 사용하여). 변수는 상태를 가집니다. 즉, 그래프를 연속해서 실행할 때 변수는 동일한 값을 유지합니다. 일반적으로 변수는 모델 파라미터를 저장하는 데 사용하지만 다른 목적으로도 쓰입니다(예를 들면 전체 훈련 스텝을 카운트하기 위해).
- 플레이스 홀더는 기술적으로 봤을 때 많은 일을 하지 않습니다. 표현하려면 텐서의 크기와 타입에 관한 정보를 가지고 있을 뿐 아무런 값도 가지고 있지 않습니다. 실제로 플레이스 홀더에 의존하고 있는 연산을 평가하려면 플레이스 홀더의 값을 텐서플로에 제공해야 합니다. 그렇지 않으면 예외가 발생할 것입니다. 일반적으로 플레이스 홀더는 실행 단계에서 텐서플로에 훈련 데이터와 테스트 데이터를 주입하기 위해 사용됩니다. 또한 변수의 값을 바꾸지 위해 할당 연산 노드에 값을 전달하는 용도로도 사용됩니다.

#### 8. 플레이스 홀더에 의존하는 연산을 평가하기 위해 그래프를 실행할 때 플레이스 홀더에 값을 주입하지 않으면 어떻게 될까요? 플레이스 홀더에 의존하지 않는 연산이라면 어떻게 될까요?
- 플레이스 홀더에 의존하는 연산을 평가할 때 플레이스 홀더에 값을 주입하지 않으면 예외가 발생합니다. 플레이스 홀더에 의존하지 않는 연산이라면 예외가 발생하지 않습니다.

#### 9. 그래프를 실행할 때 어떤 연산자의 출력값을 주입할 수 있나요? 아니면 플레이스 홀더의 값만 가능한가요?
- 그래프를 실행할 때 플레이스 홀더뿐만 아니라 어떤 연산의 출력값도 주입할 수 있습니다. 그러나 실제로는 이는 매우 드문 경우입니다(예를 들어 동결된 층의 출력을 캐싱할 때 사용할 수 있습니다. 11장 참조).

#### 10. (실행 단계에서) 변수에 원하는 값을 어떻게 설정할 수 있나요?
- 그래프를 구성할 때 변수의 초기화 값을 지정할 수 있고, 나중에 실행 단계에서 변수의 초기화 함수를 실행할 때 초기화될 것입니다. 실행 단계에서 변수의 값을 변경하는 간단한 방법은 (구성 단계에서) tf.assign()을 이용한 할당 노드를 만들고 매개변수로 변수와 플레이스 홀더에 전달하는 것입니다. 그리고 실행 단계에서 플레이스 홀더를 사용해 변수에 새로운 값을 주입하여 할당 연산을 실행합니다.

#### 11. 후진 모드 자동 미분으로 변수 10개에 대한 비용 함수의 그래디언트를 계산하려면 그래프를 몇 번 순회해야 하나요? 전진 모드 자동 미분이나 기호 미분의 경우는 어떨까요?
- 후진 모드 자동 미분은 변수 개수에 상관없이 변수에 대한 비용 함수의 그래디언트를 계산하기 위해 그래프를 두 번 순회해야 합니다.
- 전진 모드 자동 미분은 각 변수마다 한 번씩 실행해야 합니다(그러므로 10개의 다른 변수에 대한 그래디언트를 계산하려면 10번 실행).
- 기호 미분에서는 그래디언트를 계산하기 위해 다른 그래프를 만듭니다. 그래서 원본 그래프를 순회하지 않습니다(그래디언트를 위한 새 그래프를 만들 때 제외). 최적화가 매우 잘된 기호 미분 시스템은 모든 변수에 대한 그래디언트를 계산하기 위해 딱 한 번 새 그래디언트를 실행할 수 있습니다. 하지만 새 그래프가 매우 복잡하고 원본 그래프에 비해 비효율적일 수 있습니다.

#### 12. 텐서플로를 사용해 미니배치 경사 하강법으로 로지스틱 회귀를 구현해보세요. moons 데이터셋에 훈련시키고 평가해보세요. 그리고 다음 부가 기능을 추가해보세요
- (손쉽게 재사용할 수 있도록) logistic_regression() 안에 그래프를 정의해보세요.
- 훈련하는 동안 saver로 체크포인트를 정기적으로 저장하세요. 그리고 훈련 마지막에 최종 모델을 저장하세요.
- 훈련이 중지되고 다시 시작될 때 마지막 체크포인트를 복원하세요.
- 텐서보드에서 그래프가 일목요연하게 보이도록 이름 범위를 사용해 그래프를 정의하세요.
- 서머리를 추가해 텐서보드에서 학습 곡선을 시각화해보세요.
- 학습률이나 미니배치 크기 같은 하이퍼파라미터를 바꿔보고 학습 곡선의 형태를 살펴보세요.